In [25]:
import pandas as pd
import requests
import time
import re
import os

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

dataIn = './../dataIn/'
dataOut = './../dataOut/'

In [4]:
# 크롬 드라이버 다운로드 :  https://googlechromelabs.github.io/chrome-for-testing/
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간 10초
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [5]:
# diver.maximize_window()

In [6]:
# 스타벅스 음료 목록 페이지
starbucks_beverage_url = 'https://www.starbucks.co.kr/menu/drink_list.do'
driver.get(starbucks_beverage_url)

In [7]:
# 스타벅스 음료 페이지의 html 코드를 파싱해서 html 파일에 기록합니다.
html = driver.page_source # 해당 페이지 소스 코드 반환
filename = dataOut + 'starbucks_beverage.html'
htmlFile = open(filename, mode='wt', encoding='UTF-8')
print(html, file=htmlFile)
htmlFile.close()
print(filename, '파일 생성됨')

./../dataOut/starbucks_beverage.html 파일 생성됨


In [8]:
image_info = [] # (출처, alt속성, 상품코드)형식의 tuple을 담고 있는 list

try:
    # 음료 메뉴 이미지 로딩 대기하기
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'img')))

    time.sleep(3)

    # 모든 img 태그 추출
    img_elements = driver.find_elements(By.TAG_NAME, 'img')
    print(f'이미지 개수 : {len(img_elements)}')

    for img in img_elements:
        src = img.get_attribute('src')

        # 특정한 경로( 'skuimg' 단어가 포함된 )로 시작하는 이미지만 필터링
        if src and 'skuimg' in src: # None이 아닌 문자열 중에서 'skuimg' 단어가 포함된 ...
            alt = img.get_attribute('alt')

            # 상품 코드 추출 : 정규 표현식으로 [ ] 사이의 숫자 추출
            # r은 raw string ( 눈에 보이는대로 해석하라는 뜻 ) ( ex : \\\\ = 역슬래시 2개, r'\\\\ = 역슬래시 4개)
            # (.*?) : 공백 문자를 제외한 모든 글자의 최소 매칭
            # [(.*?)\] : [ ] 안의 문자열 다 가져옴
            match = re.search(r'\[(.*?)\]', src)

            # 조건표현식 : 자바의 삼항 연산자와 유사
            product_code = match.group(1) if match else None

            image_info.append((src, alt, product_code))
        # end if
    # end for

except Exception as err:
    print(err)
finally:
    pass # driver.quit()

이미지 개수 : 334


In [9]:
# 데이터 프레임으로 변환
df = pd.DataFrame(image_info, columns=['이미지Url', '상품명', '상품코드'])
print(f'총{len(df)}개')

# csv 파일로 저장
csv_filename = dataOut + 'starbucks_beverage_images.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

총205개


In [10]:
print('# 상위 몇 개만 보기')
print(df.head())

# 상위 몇 개만 보기
                                              이미지Url          상품명  \
0  https://image.istarbucks.co.kr/upload/store/sk...  나이트로 바닐라 크림   
1  https://image.istarbucks.co.kr/upload/store/sk...   나이트로 콜드 브루   
2  https://image.istarbucks.co.kr/upload/store/sk...     돌체 콜드 브루   
3  https://image.istarbucks.co.kr/upload/store/sk...     리저브 나이트로   
4  https://image.istarbucks.co.kr/upload/store/sk...    리저브 콜드 브루   

            상품코드  
0  9200000002487  
1  9200000000479  
2  9200000002081  
3  9200000002407  
4  9200000002093  


In [11]:
# 이미지를 저장할 폴더 생성
save_image_folder = 'drink_images'
os.makedirs(save_image_folder, exist_ok=True) # 이미 존재한다면 무시함

# 스타벅스 각 음료 세부 정보 url
starbucks_beverage_detail_url = 'https://www.starbucks.co.kr/menu/drink_view.do?product_cd='

# 변수 image_info 리스트를 반복하면서 상세 URL 생성
detail_urls = []

In [12]:
for img in image_info:
    img_url = img[0] # 이미지의 url주소
    img_name = img[1] # 음료 이름

    if img_url and img_name:
        # 운영체제에서 파일 이름으로 사용하지 못하는 글자에 유의
        safe_name = ''.join(name for name in img_name if name.isalnum() or name in [' _-'])
        # isalnum : 어떤 문자나 정수인지 판별합니다
        file_path = os.path.join(save_image_folder, f'{safe_name}.jpg')

        try: # http Get 방식 요청
            response = requests.get(img_url)
            if response.status_code == 200: # 정상적으로 응답 받음
                # bite image를 파일로 저장하기
                with open(file_path, 'wb') as f: # 'wb'는 하드 디스크에 바이너리로 다운로드함
                    f.write(response.content)
                # end with
                # print(f'저장 완료 : {file_path}')
            else:
                print(f'다운로드 실패 : {response.status_code}, URL : {img_url}')
            # end if

        except Exception as err:
            print(f'예외 발생 : {err}, URL : {img_url}')
        # end try

        product_code = img[2] # 상품 번호
        if product_code:
            detail_url = starbucks_beverage_detail_url + product_code
            detail_urls.append(detail_url)
        # end if
    # end if
# end for

print('작업이 완료 되었습니다.')

작업이 완료 되었습니다.


In [13]:
print('# 상위 몇개만 결과 출력하기')
for url in detail_urls[:5]:
    print(url)
# end for

# 상위 몇개만 결과 출력하기
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000002487
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000000479
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000002081
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000002407
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000002093


In [14]:
# all_tables_data = []

In [15]:
# table_elements = driver.find_elements(By.TAG_NAME, 'table')
# print(f'개수 : {len(table_elements)}')

In [16]:
for url in detail_urls[:1]: # 첫번째 페이지에 대하여
    driver.get(url)
    html = driver.page_source # 해당 페이지 소스 코드 반환
    filename = dataOut + 'first_detail_page.html'
    htmlFile = open(filename, mode='wt', encoding='UTF-8')
    print(html, file=htmlFile)
    htmlFile.close()
    print(filename, '파일 생성됨')
# end for

./../dataOut/first_detail_page.html 파일 생성됨


In [69]:
# 모든 상세 페이지를 반복하면서 유용한 데이터 수집
all_product_data = [] # 모든 테이블 데이터를 담을 리스트

In [70]:
for idx in range(len(detail_urls)):
    print(f'{idx+1}/{len(detail_urls)}번째 페이지 작업 중입니다.')
    driver.get(detail_urls[idx]) # 해당 페이지로 이동
    html = driver.page_source # html 소스 읽기
    soup = BeautifulSoup(html, 'html.parser')

    name = soup.select_one('.product_view_detail .myAssignZone h4')
    name_kr = name.contents[0].strip()
    name_en = name.find('span').get_text(strip=True)
    description = soup.select_one('.product_view_detail .myAssignZone p').get_text(strip=True)
    nutrition_info = soup.select_one('#product_info01 > p').get_text(strip=True)

    nutrition_data = {}
    # items : 영양 정보 아래에 있는 모든 <dt> 태그 목록
    items = soup.select('div.product_info_content dt')

    for onedt in items:
        key = onedt.get_text(strip=True)
        value = onedt.find_next('dd').get_text(strip=True)
        nutrition_data[key] = value # 사전에 담기
    # end for

    print(f'{nutrition_info}')

    # 모든 데이터를 하나로 합칩니다.
    sub_dict = {
        '한글 이름' : name_kr,
        '영문 이름' : name_en,
        '제품 설명' : description,
        '제품 영양 정보' : nutrition_info,
        'nutrition_data' : nutrition_data
    }

    all_product_data.append(sub_dict)
# end for

1/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
2/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
3/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
4/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
5/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
6/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
7/205번째 페이지 작업 중입니다.
Grande(그란데) /473ml
8/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
9/205번째 페이지 작업 중입니다.

10/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
11/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
12/205번째 페이지 작업 중입니다.
Bottle(보틀) /500ml
13/205번째 페이지 작업 중입니다.

14/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
15/205번째 페이지 작업 중입니다.
Grande(그란데) /473ml
16/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
17/205번째 페이지 작업 중입니다.
Trenta(트렌타) /887ml
18/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
19/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
20/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
21/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
22/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
23/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
24/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
25/205번째 페이지 작업 중입니다.
Tall(톨) /355ml
26/205번째 페이지 작업 중입니다.
Solo(솔로) /22ml
27/205번째 페이지 작업 중입니다.
Solo(솔로) /22ml
28/205번째 페이지 작업 중입니다.



In [68]:
all_product_data

[{'한글 이름': '나이트로 바닐라 크림',
  '영문 이름': 'Nitro Vanilla Cream',
  '제품 설명': '부드러운 목넘김의 나이트로 커피와 바닐라 크림의 매력을 한번에 느껴보세요!',
  '제품 영양 정보': 'Tall(톨) /355ml',
  'nutrition_data': {'1회 제공량 (kcal)': '80',
   '포화지방 (g)': '2',
   '단백질 (g)': '1',
   '지방 (g)': '2.7',
   '트랜스지방 (g)': '0',
   '나트륨 (mg)': '40',
   '당류 (g)': '10',
   '카페인 (mg)': '232',
   '콜레스테롤 (mg)': '5',
   '탄수화물 (g)': '10'}}]

In [46]:
# dict , list , tuple 공부 . ( MakeDictExam 파일 참조 )

In [71]:
all_product_data[0].keys()

dict_keys(['한글 이름', '영문 이름', '제품 설명', '제품 영양 정보', 'nutrition_data'])

In [73]:
correct_products = [] # 중첩 사전이 보정된 list

for item in all_product_data:
    base_info = {
        '한글 이름' : item['한글 이름'],
        '영문 이름' : item['영문 이름'],
        '제품 설명' : item['제품 설명'],
        '제품 영양 정보' : item['제품 영양 정보']
    }
    # 중첩 사전 형식의 영양 정보
    nutrition = item['nutrition_data']

    # dictionary unpacking : 중첩된 사전 정보를 풀어 헤쳐서 단순사전 형식으로 만들어 주는 기법
    combined = {**base_info, **nutrition}

    correct_products.append(combined)
# end for

df_product = pd.DataFrame(correct_products)

print(type(df_product.columns)) # Series

# 문자열 Series에는 문자열 조작 관련 전용 속성 ".str"이 자동으로 들어 있습니다.
# 모든 문자열 관련 함수들을 사용할 수 있습니다.
df_product.columns = df_product.columns.str.replace(' (','(', regex=False)
df_product.columns

<class 'pandas.core.indexes.base.Index'>


Index(['한글 이름', '영문 이름', '제품 설명', '제품 영양 정보', '1회 제공량(kcal)', '포화지방(g)',
       '단백질(g)', '지방(g)', '트랜스지방(g)', '나트륨(mg)', '당류(g)', '카페인(mg)',
       '콜레스테롤(mg)', '탄수화물(g)'],
      dtype='object')

In [75]:
csv_filename = dataOut + 'df_product.csv'
df_product.to_csv(csv_filename, index=False, encoding='utf-8-sig')
print(f'{csv_filename} 파일 저장 완료')

./../dataOut/df_product.csv 파일 저장 완료


In [ ]:
# 분석할 것
# '카페인' 용량이 가장 많은/적은 품목
# '제품 영양 정보' 데이터 분리